In [1]:
!pip install  evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ef12adf0f6f6331ac458ed20d7c0191c3b9e7c6498b175eb9ffa0f420c055cac
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#load the distilled model
tokenizer = AutoTokenizer.from_pretrained("NuppuCat/distillBart-6-6-1000xsum-8epoche")
student = AutoModelForSeq2SeqLM.from_pretrained("NuppuCat/distillBart-6-6-1000xsum-8epoche").to(device)


# 1. 加载教师模型和学生模型
#load the teacher model
teacher_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)
# 教师模型只用于生成软标签，因此设为eval模式，不进行反向传播
teacher_model.eval()
student.eval()

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      

In [3]:
# function to calculate the model size
def get_model_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())  # 计算参数占用
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())  # 计算缓冲区占用
    total_size = param_size + buffer_size  # 总大小（字节）
    return total_size / (1024 ** 2)  # 转换为 MB


In [4]:
import evaluate
from datasets import load_dataset
from tqdm import tqdm
# 加载 XSum 测试集前 20 个样本
#This time test 200 samples
#First test the performance of the distilled model
test_dataset = load_dataset("EdinburghNLP/xsum", split="test[:200]")

# ROUGE 计算器
rouge = evaluate.load("rouge")

# 生成摘要并收集结果
generated_summaries = []
reference_summaries = []

for sample in tqdm(test_dataset):
    article = sample["document"]
    reference_summary = sample["summary"]

    # 编码输入
    inputs = tokenizer(
        article,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding="max_length"
    ).to(device)

    # 生成摘要
    summary_ids = student.generate(
        **inputs,
        max_length=400,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # 解码生成的摘要
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

# 计算 ROUGE 分数
results = rouge.compute(
    predictions=generated_summaries,
    references=reference_summaries,
    use_stemmer=True
)

# 打印结果
print('Student model')
print(f"Model size: {get_model_size(student):.2f} MB")
for key in results:
    print(f"{key}: {results[key]*100:.2f}")

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1527: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
100%|██████████| 200/200 [01:01<00:00,  3.26it/s]


Student model
Model size: 877.32 MB
rouge1: 30.72
rouge2: 9.68
rougeL: 23.79
rougeLsum: 23.77


In [5]:
import evaluate
from datasets import load_dataset
from tqdm import tqdm
# 加载 XSum 测试集前 20 个样本
test_dataset = load_dataset("EdinburghNLP/xsum", split="test[:200]")
#Use same test model to test the teacher model
# ROUGE 计算器
rouge = evaluate.load("rouge")

# 生成摘要并收集结果
generated_summaries = []
reference_summaries = []

for sample in tqdm(test_dataset):
    article = sample["document"]
    reference_summary = sample["summary"]

    # 编码输入
    inputs = tokenizer(
        article,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding="max_length"
    ).to(device)

    # 生成摘要
    summary_ids = teacher_model.generate(
        **inputs,
        max_length=400,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # 解码生成的摘要
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

# 计算 ROUGE 分数
results = rouge.compute(
    predictions=generated_summaries,
    references=reference_summaries,
    use_stemmer=True
)

# 打印结果
print('teacher_model')
print(f"Model size: {get_model_size(teacher_model):.2f} MB")
for key in results:
    print(f"{key}: {results[key]*100:.2f}")

100%|██████████| 200/200 [03:48<00:00,  1.14s/it]


teacher_model
Model size: 1550.07 MB
rouge1: 20.81
rouge2: 3.67
rougeL: 13.45
rougeLsum: 13.43


In [6]:
import evaluate
from datasets import load_dataset
from tqdm import tqdm
# Use the same testset to test the original student model 
student_model_original = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-6-6").to(device)
# 加载 XSum 测试集前 20 个样本
test_dataset = load_dataset("EdinburghNLP/xsum", split="test[:200]")

# ROUGE 计算器
rouge = evaluate.load("rouge")

# 生成摘要并收集结果
generated_summaries = []
reference_summaries = []

for sample in tqdm(test_dataset):
    article = sample["document"]
    reference_summary = sample["summary"]

    # 编码输入
    inputs = tokenizer(
        article,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding="max_length"
    ).to(device)

    # 生成摘要
    summary_ids = student_model_original.generate(
        **inputs,
        max_length=400,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # 解码生成的摘要
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

# 计算 ROUGE 分数
results = rouge.compute(
    predictions=generated_summaries,
    references=reference_summaries,
    use_stemmer=True
)

# 打印结果
print('student_model_original')
print(f"Model size: {get_model_size(student_model_original):.2f} MB")
for key in results:
    print(f"{key}: {results[key]*100:.2f}")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

100%|██████████| 200/200 [02:12<00:00,  1.51it/s]


student_model_original
Model size: 877.32 MB
rouge1: 20.28
rouge2: 3.45
rougeL: 13.33
rougeLsum: 13.31


In [7]:
#
#At last, I want to compare the directly fine tune the student model
# 2. 加载并预处理数据集
from datasets import load_dataset
# 加载 XSUM 的 1000 个样本
xsum_dataset = load_dataset("EdinburghNLP/xsum")
dataset  = xsum_dataset["train"].shuffle(seed=42).select(range(1000))
# 定义预处理函数，将输入文本和摘要转为token id
def preprocess(example):
    # 输入为完整文章（字段名可能是 "document"），目标为摘要（字段名为 "summary"）
    inputs = tokenizer(example["document"], truncation=True, padding="max_length", max_length=1024)
    targets = tokenizer(example["summary"], truncation=True, padding="max_length", max_length=400)
    # 注意：生成任务需要 labels 字段
    labels = targets["input_ids"]
    labels = [ [t if t != tokenizer.pad_token_id else -100 for t in label] for label in labels]
    
    inputs["labels"] = labels
    return inputs
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
# 4. 设置训练参数
#With same training args
training_args = Seq2SeqTrainingArguments(
   
    output_dir="./fintune_results",
    per_device_train_batch_size=4,
    num_train_epochs=7,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=student_model_original)
trainer = Seq2SeqTrainer(
    model=student_model_original,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 开始训练
trainer.train()

<ipython-input-10-7a3eaf41d7de>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,3.317400
20,2.858300
30,2.879200
40,2.537700
50,2.678000
60,2.594800
70,2.465200
80,2.624900
90,2.617800
100,2.605000


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarnin

TrainOutput(global_step=875, training_loss=0.9988510544640677, metrics={'train_runtime': 1451.8736, 'train_samples_per_second': 4.821, 'train_steps_per_second': 0.603, 'total_flos': 7584954187776000.0, 'train_loss': 0.9988510544640677, 'epoch': 7.0})

In [13]:
import evaluate
from datasets import load_dataset
from tqdm import tqdm
# 加载 XSum 测试集前 20 个样本
# From the Training loss, we can see that the fine tune is almost done, as the trainset with only 1000 samples, 7 epoche is good enough.
# Finally, test the fine-tuned student model on the same testset.
test_dataset = load_dataset("EdinburghNLP/xsum", split="test[:200]")

# ROUGE 计算器
rouge = evaluate.load("rouge")

# 生成摘要并收集结果
generated_summaries = []
reference_summaries = []

for sample in tqdm(test_dataset):
    article = sample["document"]
    reference_summary = sample["summary"]

    # 编码输入
    inputs = tokenizer(
        article,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding="max_length"
    ).to(device)

    # 生成摘要
    summary_ids = student_model_original.generate(
        **inputs,
        max_length=400,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # 解码生成的摘要
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    generated_summaries.append(generated_summary)
    reference_summaries.append(reference_summary)

# 计算 ROUGE 分数
results = rouge.compute(
    predictions=generated_summaries,
    references=reference_summaries,
    use_stemmer=True
)

# 打印结果
# Not bad
print('student_model_finetuned')
print(f"Model size: {get_model_size(student_model_original):.2f} MB")
for key in results:
    print(f"{key}: {results[key]*100:.2f}")

100%|██████████| 200/200 [02:13<00:00,  1.50it/s]


student_model_finetuned
Model size: 877.32 MB
rouge1: 25.49
rouge2: 7.16
rougeL: 17.83
rougeLsum: 17.85


In [ ]:
#We can see the results compare with distilled student model(Student model, NuppuCat/distillBart-6-6-1000xsum-8epoche), 
#teacher model without finetune on Xsum but finetuned on CNN dataset(teacher_model, facebook/bart-large-cnn), 
#the original student model(student_model_original, sshleifer/distilbart-cnn-6-6),
#and the fine-tuned original student model (student_model_finetuned, )
#The distilled student model reached the best performance. 